# Face Recognition using PCA

In [89]:
#Face recognition systemm using Principal Component Analysis. 

#Dataset used is available at https://www.cl.cam.ac.uk/research/dtg/attarchive/facedatabase.html
#Dataset consists of 400 facial images, 10 each of 40 different subjects.
#7 images of each subject are used for training and 3 for testing

In [90]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [118]:
#converting an image into a data matrix
def readImg(direc):
    images = []
    for i in range(1,41): #40 subjects
          for j in range(1,8): #7 training images
                #image name is s1/1.pgm for 1st image of subject 1.
                im = cv2.imread(r"{}/s{}/{}.pgm".format(direc,i,j),0)
                im=im.flatten()  #nxn to n2x1 matrix
                images.append(im)
    images=np.array(images)
    images=images.astype(np.float64)
    return images



# Add image directory here

In [119]:
# images = readImg("Directory")
images = readImg("img")
#print(images)

In [120]:
#PCA
#calculating mean face, i.e average of all faces
mean_face=images.sum(axis=0)
n_img=len(images)
for i in range(len(mean_face)):
    mean_face[i]/=n_img

#subtracting the mean face from all faces
avg_images=[]
for img in images:
    avg_images.append(img-mean_face)
#print(avg_images)

In [121]:
#covariance matrix
A=np.array(avg_images).T
C=np.matmul(A.T,A)
#print(C)

In [122]:
#Computing eigenfaces

#compute the eigenvectors ui of A_trans.A
e,u=np.linalg.eig(C)
v=np.matmul(A,u)
v_n=[]
for i in range(len(v.T)) :
    v_n.append(v.T[i]/np.linalg.norm(v.T[i]))

# k=12, k highest eigenvalues,eigenvectors
k=12
eig=e[0:k]
vec=v_n[0:k]
wi=np.matmul(vec,np.array(avg_images).T)
W=wi.T

In [129]:
#Representing faces onto the basis of eigenfaces
#wi=np.matmul(vec,np.array(avg_images).T)
#W=wi.T    #weights of faces in training dataset

#input i,j represents jth image of ith subject to be tested
def face_recog(direc,i,j):
    f= cv2.imread(r"{}/s{}/{}.pgm".format(direc,i,j),0)
    f=f.flatten()
    dist=[]
    avg_face=f-mean_face
    wi=np.matmul(vec,np.array(avg_face).T)
    for i in range(len(W)):
        dist.append(np.linalg.norm(wi-W[i])) #distances between training images and test image
    d=dist.index(min(dist))   #image closest to test image
    if (min(dist)<2800):      #threshold
        print("The given image is of subject number",int(d/7+1))  #subject number
        return int(d/7+1)
    else:
        print("Unidentified face")
        return 0
    

# Input test data here:

In [130]:
#face_recog(directory,subject number, image number)
face_recog('img',26,10)

The given image is of subject number 26


26

In [128]:
#accuracy

count=0
for i in range(1,41):
    for j in range(8,11):
        d=face_recog('img',i,j)
        if(d==i):
            count+=1
print(count)       
        

114
